In [19]:
from llmcoder import LLMCoder

In [20]:
llmcoder = LLMCoder(analyzers=["mypy_analyzer_v1"])

In [21]:
code = '''from llmcoder import LLMCoder

# Create an LLMCoder with a random seed of 0
llmcoder = '''

In [22]:
result = llmcoder.complete(code)

Creating first completion...


Starting feedback loop...
Starting feedback iteration 1...
Analyzing code...
Running MypyAnalyzer...
No missing stubs found.
0 / 1 analyzers passed
Starting feedback iteration 2...
Analyzing code...
Running MypyAnalyzer...
No missing stubs found.
1 / 1 analyzers passed


In [23]:
result

'LLMCoder()'

In [24]:
import re

In [25]:
for message in llmcoder.messages:
    print(message['role'].upper())
    # Remove colors from the content of the message
    message['content'] = re.sub(r'\x1b[^m]*m', '', message['content'])
    print(message['content'])
    print('-' * 80)

SYSTEM
You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.
You are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.
Precisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.
Your code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.
Your unique capability is to provide completions without altering, repeating, or commenting on the original code.
You offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.
This approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl